In [36]:
from keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, MaxPooling2D, Conv2D
import os
import pandas as pd
import numpy as np

In [37]:
TRAIN_DIR = 'images/train'
TEST_DIR = 'images/validation'

In [38]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir, label)):
            image_paths.append(os.path.join(dir, label, imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths, labels

In [39]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [40]:
print(train)

                                image     label
0            images/train\angry\0.jpg     angry
1            images/train\angry\1.jpg     angry
2           images/train\angry\10.jpg     angry
3        images/train\angry\10002.jpg     angry
4        images/train\angry\10016.jpg     angry
...                               ...       ...
28816  images/train\surprise\9969.jpg  surprise
28817  images/train\surprise\9985.jpg  surprise
28818  images/train\surprise\9990.jpg  surprise
28819  images/train\surprise\9992.jpg  surprise
28820  images/train\surprise\9996.jpg  surprise

[28821 rows x 2 columns]


In [41]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TRAIN_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [42]:
from tqdm.notebook import tqdm

In [43]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, grayscale = True)
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), 48, 48,1)
    return features
    

In [44]:
train_features = extract_features(train['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

C:\Users\angad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\utils\image_utils.py:409: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn(


In [45]:
test_features = extract_features(test['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

In [46]:
x_train  = train_features/255.0
x_test = test_features/255.0

In [47]:
from sklearn.preprocessing import LabelEncoder

In [48]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [49]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [50]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [51]:
model = Sequential()
#Conv Layers

model.add(Conv2D(128, kernel_size=(3,3), activation = "relu", input_shape = (48, 48, 1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation = "relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation = "relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation = "relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())

# second layers
model.add(Dense(512, activation = "relu"))
model.add(Dropout(0.4))
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.3))

#output layer
model.add(Dense(7, activation='softmax'))

In [52]:
model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = "accuracy")

In [53]:
model.fit(x= x_train, y= y_train, batch_size = 128, epochs = 100, validation_data = (x_test, y_test))

Epoch 1/100
226/226 [==============================] - 202s 891ms/step - loss: 1.8228 - accuracy: 0.2446 - val_loss: 1.8116 - val_accuracy: 0.2486
Epoch 2/100
155/226 [===================>..........] - ETA: 54s - loss: 1.7992 - accuracy: 0.2525

KeyboardInterrupt: 

In [65]:
from keras.models import model_from_json

In [66]:
# model_json = model.to_json()
# with open("emotiondetector.json", "w") as json_file:
#     json_file.write(model_json)
# model.save("emotiondetector.h5")

In [67]:
json_file = open("emotiondetector.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("emotiondetector.h5")

In [68]:
label = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [69]:
def ef(image):
    img = load_img(image, grayscale = True)
    feature = np.array(img)
    feature = feature.reshape(1,48,48,1)
    return feature/255.0

In [71]:
image = 'images/train/disgust/299.jpg'
print("original image: disgust")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("Model prediction", pred_label)

original image: disgust
1/1 [==============================] - 0s 19ms/step
Model prediction disgust
